In [1]:
#Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
import os
import requests
# from urllib.parse import urlparse   
import numpy as np
# To read the PDF
import PyPDF2
# To analyze the PDF layout and extract text
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# To extract text from tables in PDF
import pdfplumber
import spacy
# To extract the images from the PDFs
from PIL import Image
from pdf2image import convert_from_path
# To perform OCR to extract text from images 
# To remove the additional created files
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
import os
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, unquote
import textwrap
from selenium import webdriver


In [ ]:
# !pip install selenium


In [2]:
nlp = spacy.load("en_core_web_sm")

/Users/davidoluyalegbenga/.pyenv/versions/3.9.6/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


<h3>Download Text Contents - SEH</h3>
<p>Use this to scrap needed text from a web source for SEH</p>


In [3]:
def extract_link(row):
    # Check if 'Link' column is available in the DataFrame
    if 'Link' in row.index:
        link = row['Link']
    else:
        link = None
    
    # Function to extract link from 'Title (+link)*' column
    if 'Title (+link)*' in row.index and pd.notna(row['Title (+link)*']):
        title_link_str = row['Title (+link)*']
    else:
        title_link_str = ""
    
    if isinstance(link, str):
        return link
    else:
        # If 'Link' is not a string, extract link from 'Title (+link)*' column
        match = re.search(r'\((.*?)\)', title_link_str)
        if match:
            return match.group(1)
        else:
            return None


In [28]:
def scrap_webpage(metadata_file_list, folder_names):
    for metadata_file_index, metadata_file in enumerate(metadata_file_list):
        metadata_df = pd.read_excel(f'../01_Input/00_Metadata/{metadata_file}')
        metadata_df['Link'] = metadata_df.apply(extract_link, axis=1)  # Apply extract_link function
        metadata_df['Scraped'] = 'No'  # Add 'Scraped' column to indicate whether a page has been scraped
        
        for folder_name_index, folder_name in enumerate(folder_names):
            if not os.path.exists(folder_name):
                os.makedirs(folder_name)
            
            if metadata_file_index == folder_name_index:
                for index, row in metadata_df.iterrows():
                    if row['Scraped'] == 'Yes':
                        continue  # Skip if already scraped
                    
                    if ('Link' in row.index and pd.notna(row['Link'])) or ('Title (+link)*' in row.index and pd.notna(row['Title (+link)*'])):
                        # Get the URL to scrape
                        web_link = row['Link'] if 'Link' in row.index and pd.notna(row['Link']) else row['Title (+link)*']
                        # Extract the file extension
                        file_extension = os.path.splitext(web_link)[1]

                        # Check if the file extension is .pdf
                        if file_extension.lower() != '.pdf':

                            try:
                                headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

                                response = requests.get(web_link, headers=headers, allow_redirects=True, timeout=10)
                                
                                title = row['Title']
                                # print(web_link)
                                # print(response.status_code)
                                print('______________')
                                if response.status_code == 200:
                                    # Parse the HTML
                                    soup = BeautifulSoup(response.content, "html.parser")
                                    # Extract text from specific HTML elements
                                    text = soup.get_text(separator="\n")
                                    # Clean up the text
                                    text = re.sub(r'\n+', '\n', text)  # Remove consecutive newlines
                                    text = text.strip()  # Remove leading and trailing whitespace
                                    text = textwrap.fill(text, width=80)  # Wrap text to 80 characters per line
                                    
                                    # Save extracted text in a text file
                                    # filename = f"{title.replace('/', '_')}_{index}.txt"
                                    filename = f"{str(title).replace('/', '_')}.txt"  # Convert title to string and replace '/'

                                    file_path = os.path.join(folder_name, filename)
                                    with open(file_path, "w", encoding="utf-8") as file:
                                        file.write(text)
                                    
                                    print(f"Content saved to: {file_path}")

                                    # Update 'Scraped' column in metadata DataFrame
                                    metadata_df.at[index, 'Scraped'] = 'Yes'

                                elif response.status_code == 403 :
                                    
                                    print("Download pdf from scihub ")
                                    print(web_link)

                                    # Send a GET request to the URL and retrieve the HTML content
                                    response = requests.get(web_link)
                                    html_content = response.content
                                    # Parse the HTML content using BeautifulSoup
                                    soup = BeautifulSoup(html_content, 'html.parser')
                                        
                                    # Find the div element with id "buttons"
                                    buttons_div = soup.find('div', {'id': 'buttons'})

                                    # Check if the div element is found
                                    if buttons_div:
                                        # Find the button element within the div
                                        pdf_button = buttons_div.find('button')

                                        if pdf_button:
                                            # Extract the value of the "onclick" attribute
                                            onclick_value = pdf_button.get('onclick')

                                            # Extract the PDF link from the onclick attribute value
                                            pdf_link = onclick_value.split("'")[1]

                                            print("PDF Link:", pdf_link)
                                        else:
                                            print("No button element found within the div.")
                                    else:
                                        print("No div element with id 'buttons' found.")


                                else:
                                    print("Failed to fetch the page.")
                            except requests.exceptions.RequestException as e:
                                print(f"Error fetching the page: {e}")

                        else :
                            print('is a pdf. Scrape differently...') 
                            #Todo:: add pdf extract function here.

                break  # Break after processing the corresponding folder


In [ ]:
# List of metadata Excel files
metadata_file_list = ['(Old) Mapping Journal articles - UNV Sustainable Energy Hub.xlsx',
                      'Energy Document Catalogue.xlsx',
                      'Energy Journal Article Compilation.xlsx',
                      'UNDP Energy CO Press Releases.xlsx']

# Define folder names array
publication_folder = "../02_Output/00_Extracted-Text/02_Publications/UNDP"
folder_names = [
                "../02_Output/00_Extracted-Text/00_SEH",
                publication_folder,
                publication_folder,
                publication_folder]



# metadata_file_list = ['(Old) Mapping Journal articles - UNV Sustainable Energy Hub.xlsx']
# folder_names = ["../02_Output/00_Extracted-Text/00_SEH"]
# Call scrap_webpage function
scrap_webpage(metadata_file_list, folder_names)

<h3> Extract National Documents & United Resolution PDFs (Open Metadata) </h3>

In [30]:
#Load Metadata CSV
metadata_df = pd.read_excel('../01_Input/00_Metadata/National Energy Document Compilation.xlsx')
metadata_df=metadata_df[metadata_df["Exists?"]=="Y"]


#Load Un Resolutions data
metadata_df_un_resolution = pd.read_excel('../01_Input/00_Metadata/UN Resolutions.xlsx')


### Clean the Dataframe
- Some rows in the .xls df don't have pdf documents. They should be skipped for now to avoid NANs

In [31]:
#If the cell doesn't have a pdf Link, skip it 
metadata_df = metadata_df.dropna(subset=['Link'])
metadata_df.dropna(subset=['Link'], inplace=True)
metadata_d = metadata_df.head(100)


#Do same for UN Resolution
metadata_df_un_resolution = metadata_df_un_resolution.dropna(subset=['Link'])
metadata_df_un_resolution.dropna(subset=['Link'], inplace=True)
# metadata_df_un_resolution = metadata_df_un_resolution.head(25)
metadata_df_un_resolution

,Resolution No,Link
0,A/C.2/77/L.70_x000D_,https://undocs.org/A/C.2/77/L.70_x000D_
1,A/RES/75/211,https://undocs.org/A/RES/75/211
2,A/78/201,https://undocs.org/A/78/201
3,A/RES/65/151,https://undocs.org/A/RES/65/151
4,A/RES/67/215,https://undocs.org/A/RES/67/215
5,A/RES/69/225,https://undocs.org/A/RES/69/225
6,A/67/150,https://undocs.org/A/67/150
7,A/69/395,https://undocs.org/A/69/395
8,A/RES/63/210,https://undocs.org/A/RES/63/210
9,A/RES/66/206,https://undocs.org/A/RES/66/206


<h3> Donwload PDFs </h3>

In [32]:
#download a pdf from a url to a path if is exists
def download_document(pdf_url,pdf_path):
    
    # Code to download the document from the URL
    # Handle different formats (PDF, HTML, etc.)
   
    try:
        response = requests.get(pdf_url,timeout=8)

        if response.status_code == 200:
            #content_disposition = response.headers.get("content-disposition")

            with open(pdf_path, "wb") as file:
                file.write(response.content)

            print(f"{pdf_path} successfully downloaded document with path: {pdf_url}")
            return pdf_path
        else:
            print(f"Code 200, failed to downloaded document with path: {pdf_url}")
            return "None"
    except:
        print(f"Failure to download document with path: {pdf_url}")
        return "None"
    
#Example Usage


In [33]:
#Download missing document
def download_document_if_missing(row,pdf_path):
    if not os.path.exists(pdf_path):
        ##Download document if the pdf is not already present
        pdf_url = row['Link']
        if not pd.isna(pdf_url):
            download_document(pdf_url,pdf_path)
        else:
            print(pdf_path,"no pdf or link")
    else:
        print(pdf_path,"already exists")
        
#Example Usage


In [34]:
#Run the download function for all
def download_all(metadata_df,missing_only=True):
    
    for index, row in metadata_df.iterrows():

        #nationals 
        folder="../01_Input/01_PDFs/03_National/"+row["Code"].split("-")[0]
        # Check if the directory already exists
        if not os.path.exists(folder):
            # Create the directory
            os.makedirs(folder)
            print(folder+"directory created!")

        #nationals
        pdf_path=folder+'/'+row["Code"]+".pdf"

        #un resolution
        pdf_path=folder+'/'+row["Resolution No"]+".pdf"

        download_document_if_missing(row,pdf_path)


In [35]:
#Run the download function for all
def download_all_un_resolution(metadata_df,missing_only=True):
    
    for index, row in metadata_df.iterrows():

        folder="../01_Input/01_PDFs/05_UN Resolutions/"+row["Resolution No"].split("/")[0]
        pdf_link = row['Link']

        # Initialize a webdriver (make sure you have chromedriver or geckodriver installed and in your PATH)
        driver = webdriver.Chrome()  # Or specify the path to your webdriver executable
        # Open the webpage
        driver.get(pdf_link)

        # Get the page source after the JavaScript has been executed
        html_content = driver.page_source

        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(html_content, "html.parser")

        # Find the link element with inner text "English"
        link = soup.find("a", text="English")
        if link:
            # Extract the URL from the href attribute
            url = link["href"]
            print("URL:", url)
             # Initialize a webdriver (make sure you have chromedriver or geckodriver installed and in your PATH)
            driver = webdriver.Chrome()  # Or specify the path to your webdriver executable

            # Open the URL in the browser
            driver.get(url)

            # Give some time for the download to start
            time.sleep(15)  # Adjust this time as needed
            driver.quit()

        else:
            print("Link not found.")
        # Close the browser

       
        # Check if the directory already exists
        if not os.path.exists(folder):
            # Create the directory
            os.makedirs(folder)
            print(folder+"directory created!")
        #un resolution
        pdf_path=folder+'/'+row["Resolution No"]+".pdf"
        
        driver.quit()

 


In [ ]:
# ## Run the download pipeline for all
# download_all(metadata_df,missing_only=True)
# download_all_un_resolution(metadata_df_un_resolution,missing_only=True)

<h1> Text Extraction (PDF to text) </h1>
<pre>The text extraction flow is modified to extract in details
  - Table of content
  - Clean Page texts
  - Tables
  - Text from Images
  - Images/diagrams
  </pre>

In [36]:

def text_extraction(element):
    # Extracting the text from the in-line text element
    line_text = element.get_text()
    
    # Find the formats of the text
    # Initialize the list with all the formats that appeared in the line of text
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Iterating through each character in the line of text
            for character in text_line:
                if isinstance(character, LTChar):
                    # Append the font name of the character
                    line_formats.append(character.fontname)
                    # Append the font size of the character
                    line_formats.append(character.size)
    # Find the unique font sizes and names in the line
    format_per_line = list(set(line_formats))
    
    # Return a tuple with the text in each line along with its format
    return (line_text, format_per_line)

In [37]:
# Extracting tables from the page

def extract_table(pdf_path, page_num, table_num):
    # Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    # Find the examined page
    table_page = pdf.pages[page_num]
    # Extract the appropriate table
    table = table_page.extract_tables()[table_num]
    return table

In [38]:
# Convert table into the appropriate format
def table_converter(table):
    table_string = ''
    # Iterate through each row of the table
    for row_num in range(len(table)):
        row = table[row_num]
        # Remove the line breaker from the wrapped texts
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Convert the table into a string 
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Removing the last line break
    table_string = table_string[:-1]
    return table_string


In [39]:
#Identify table of contents in text and format it properlly
def clean_text_toc(text_content):
    # Define a regular expression pattern to match the table of contents
    toc_pattern = re.compile(r'^\s*(\w+\.\s+.+?)\s+(\d+)\s*', re.MULTILINE)

    # Find all matches in the text content
    matches = toc_pattern.findall(text_content)

    # Format the matches as a table of contents
    table_of_contents_title = "Table of Contents:\n"
    table_of_contents = ""

    isToc = False
    for item, page in matches:
        # Check if page contains dots
        if ". . . . . ."  in text_content or "........" in text_content:
            isToc = True
            # Remove dots between the title and page number
            formatted_item = item.replace(". . . .","").replace("...","")
            formatted_item = re.sub(r'\s+', ' ', formatted_item).strip()
            table_of_contents += f"{formatted_item}: Page: {page}\n"
        else : 
            continue  # Skip if it doesn't contain dots, not a table of content entry

    if isToc : 
        return table_of_contents_title + table_of_contents
    else : 
        return text_content


In [40]:
def multimodal_content_text_extractor(extract_text_path,pdf_path):
    
    # print(pdf_path)
    pdf_path=pdf_path
    # create a PDF file object
    pdfFileObj = open(pdf_path, 'rb')

    # create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdfFileObj)
    total_no_pages = len(list(pdf_reader.pages))

    # Create the dictionary to extract text from each image
    text_per_page = {}
    # We extract the pages from the PDF
    for pagenum, page in enumerate(extract_pages(pdf_path)):
        # Initialize the variables needed for the text extraction from the page
        pageObj = pdf_reader.pages[pagenum]
        page_text = []
        line_format = []
        text_from_images = []
        text_from_tables = []
        page_content = []
        # Initialize the number of the examined tables
        table_num = 0
        first_element= True
        table_extraction_flag= False
        # Open the pdf file
        pdf = pdfplumber.open(pdf_path)
        # Find the examined page
        page_tables = pdf.pages[pagenum]
        # Find the number of tables on the page
        tables = page_tables.find_tables()


        # Find all the elements
        page_elements = [(element.y1, element) for element in page._objs]
        # Sort all the elements as they appear in the page 
        page_elements.sort(key=lambda a: a[0], reverse=True)

        # Find the elements that composed a page
        for i,component in enumerate(page_elements):
            # Extract the position of the top side of the element in the PDF
            pos= component[0]
            # Extract the element of the page layout
            element = component[1]
            
            # Check if the element is a text element
            if isinstance(element, LTTextContainer):
                # Check if the text appeared in a table
                if table_extraction_flag == False:
                    # Use the function to extract the text and format for each text element
                    (line_text, format_per_line) = text_extraction(element)
                    # Append the text of each line to the page text
                    page_text.append(line_text)
                    # Append the format for each line containing text
                    line_format.append(format_per_line)
                    page_content.append(line_text)
                else:
                    # Omit the text that appeared in a table
                    pass
            
        # Create the key of the dictionary
        dctkey = 'Page_'+str(pagenum)
        # Add the list of list as the value of the page key
        text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]

    # Closing the pdf file object
    pdfFileObj.close()
    
    final_result = ""
    # print(total_no_pages)
    # Display the content of the page 0  for example
    for i in range(0, total_no_pages):
        page_text_content = ''.join(text_per_page[f"Page_{i}"][4])
        final_result += clean_text_toc(page_text_content)
       
    return textwrap.fill(final_result, width=80)

In [41]:
#Extract text from document

def extract_text_from_pdf(extract_text_path,pdf_path):
    try:
        text = multimodal_content_text_extractor(extract_text_path,pdf_path) #extract_text(pdf_path)##imported function
        ##export text
        with open(extract_text_path, 'w') as file:
            file.write(text)
    except Exception as e:
            print(f"Error processing PDF {pdf_path}: {str(e)}")
            print("invalid pdf",pdf_path)


In [42]:
#Download missing documents
def extract_text_if_missing(extract_text_path,pdf_path):
    
    if not os.path.exists(extract_text_path):
        ##extract text if it is not already present
        extract_text_from_pdf(extract_text_path,pdf_path)

    else:
        print(extract_text_path,"already exists")

In [43]:
#Run the extract function for all
def extract_all(metadata_df,missing_only=True):
    
    for index, row in metadata_df.iterrows():

        folder="03_National/"+row["Code"].split("-")[0]## update this for more docs
        pdf_path="../01_Input/01_PDFs/"+folder+'/'+row["Code"]+".pdf"
        extract_text_folder="../02_Output/00_Extracted-Text/"+folder
        
        if os.path.exists(pdf_path):
            
            if not os.path.exists(extract_text_folder):
                # Create the directory
                os.makedirs(extract_text_folder)
                print(extract_text_folder+"directory created!")

            if missing_only:
                extract_text_if_missing(extract_text_folder+'/'+row["Code"]+".txt",pdf_path)
            else:
                extract_text_from_pdf(pdf_path)
        else:
            print("can't extract because no pdf",row["Code"])

In [44]:
#Run the extract function for all
def extract_all_un_resolution(metadata_df,missing_only=True):

    # Define the folder path containing the PDF files
    folder_path = "../01_Input/01_PDFs/05_UN Resolutions"
    
    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        # Check if the file is a PDF file
        if filename.endswith(".pdf"):
            # Construct the full path to the PDF file
            pdf_path = os.path.join(folder_path, filename)
            
            # Define the path to save the extracted text
            extract_text_path = f"../02_Output/00_Extracted-Text/05_UN Resolutions/{filename}.txt"
            
            # Call the extract_text_from_pdf function with the paths
            extract_text_from_pdf(extract_text_path, pdf_path)

In [16]:
# extract_all(metadata_df) 
# extract_all_un_resolution(metadata_df_un_resolution)

<h3> Text Cleaning </h3>

In [17]:
#Text cleaning 

def remove_noise(text):
 
  # Remove lines with only whitespace characters
    cleaned_text = re.sub(r"^\s*$", "", text, flags=re.MULTILINE)
    cleaned_text = cleaned_text.replace('\uf0a7', ';')
    cleaned_text = cleaned_text.replace('\r', '\n')
    cleaned_text = re.sub(r"\n", " ", cleaned_text)  # remove newlines
    # Remove multiple consecutive newlines
    cleaned_text = re.sub(r"\n{2,}", "\n", cleaned_text)
    cleaned_text = re.sub(r"\s+", " ", cleaned_text)  # replace multiple spaces with a single space
    # cleaned_text = re.sub(r"[^a-zA-Z0-9\s]", "", cleaned_text)  # remove non-alphanumeric characters
    # Remove lines with single letters
    cleaned_text = re.sub(r"\b\w\b", "", cleaned_text) # to remove the headers . not needed.
    
    # Remove numbers followed by a dot before a phrase to remove page issues
    cleaned_text = re.sub(r"\b\d+\.\s*", "", cleaned_text).replace('.','')

    cleaned_text = re.sub(r"http\S+|www\S+|ftp\S+", "", cleaned_text) # remove urls
    
    #Remove headers taggins
    cleaned_text = re.sub(r"\b\d+-\d+\b|\b/\d+\b|\b\w/\w/\w/\b", "", cleaned_text)

    
    
    # Process the text with spaCy
    # doc = nlp(cleaned_text)
    
    # # Keep sentences and join them together
    # cleaned_text = " ".join(sent.text for sent in doc.sents)
    cleaned_text = textwrap.fill(cleaned_text, width=80) 
 
    return cleaned_text

def remove_punctuation(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    return cleaned_text

def remove_stopwords(text): 
    # nltk.download('stopwords') # only need to run this once
    stop_words = set(stopwords.words("english"))
    filtered_text = [word for word in text.split() if word.lower() not in stop_words]
    return " ".join(filtered_text)

def convert_to_lowercase(text): # reduce words to their root forms
    return text.lower()



In [18]:
def clean_text(extract_text_path,clean_text_path):
    
    with open(extract_text_path, 'r') as file:
        extract_text=file.read()
    
    # Remove unwanted characters, whitespace, etc.
    # Regular expressions can be helpful here
    # Additional cleaning steps as required
    
    clean_text = remove_noise(extract_text)
    # clean_text = remove_punctuation(text)
    # clean_text = remove_stopwords(text)
    # clean_text = convert_to_lowercase(text)
    # clean_text = lemmatize_text(text)
    
    with open(clean_text_path, 'w',  encoding="utf-8") as file:
        file.write(clean_text)

In [19]:
def clean_text_if_missing(extract_text_path,clean_text_path):
    
    if not os.path.exists(clean_text_path):
        ##extract text if it is not already present
        clean_text(extract_text_path,clean_text_path)

    else:
        print(clean_text_path,"already exists")

In [20]:
#Run the clean function for all
def clean_all(metadata_df,missing_only=True):
    
    for index, row in metadata_df.iterrows():

        folder="03_National/"+row["Code"].split("-")[0]## update this for more docs
        
        clean_text_folder="../02_Output/01_Cleaned-Text/"+folder
        extract_text_folder="../02_Output/00_Extracted-Text/"+folder
        
        extract_text_path=extract_text_folder+'/'+row["Code"]+".txt"
        clean_text_path=clean_text_folder+'/'+row["Code"]+"-clean.txt"
        
        if os.path.exists(extract_text_path):
            
            if not os.path.exists(clean_text_folder):
                # Create the
                #  directory
                os.makedirs(clean_text_folder)
                print(clean_text_folder+"directory created!")

            if missing_only:
                clean_text_if_missing(extract_text_path,clean_text_path)
            else:
                clean_text(extract_text_path,clean_text_path)
        else:
            print("can't clean because no extract",row["Code"])

In [21]:
# clean_all(metadata_df)

<h3> convert final cleaned text and metadata to jsons </h3>

In [75]:
def create_document_json(json_path,clean_text_path,row, source_type):
    print(f'current source type:: {source_type}')
    with open(clean_text_path, 'r') as file:
        text=file.read()
    
    if source_type == 'National':
        document_json = {
            'Code': str(row['Code']),
            'Status': row['Status'],
            'Country Name': row['Country Name'],
            'Country Code': str(row['Country Code']),
            'Category': row['Category'],
            'KeyWord to Search': row['KeyWord to Search'],
            'Document Title': row['Title'],
            'Exists?': row['Exists?'],
            'Category': row['Category'],
            'Publication Date': str(row['Publication Date']),
            'Publication Year': str(row['Publication Date']),
            'Start Year': str(row['Start Year']),
            'End Year': str(row['End Year']),
            'Language': row['Language'],
            'Link': row['Link'],
            'Content': text  # Extracted text content from PDF
        }

    elif source_type == 'SEH':
        document_json = {
            'Code': '',
            'Status': '',
            'Country Name': str(row['Country*']) or '' ,
            'Country Code': str(row['Country*']),
            'Category': 'SEH',
            'KeyWord to Search': '',
            'Document Title': row['Title'],
            'Exists?': 'Y',
            'Publication Date': str(row['Day of mapping']),
            'Publication Year': str(row['Year']),
            'Start Year': str(row['Year']),
            'End Year': str(row['Year']),
            'Language': 'EN',
            'Link': str(row['Title (+link)*']) or '',
            'Content': text,  # Extracted text content from PDF

            'Region': row['Region*'],
            'Journal': row['Journal'],
            'Authors': row['Authors/Institution*']

        }
        
    elif source_type == 'Publications':
        document_json = {
            'Code': '',
            'Status': '',
            'Country Name': str(row.get('Country Name', '')),
            'Country Code': str(row.get('Country Name', '')),
            'Category': 'Publications',
            'KeyWord to Search': str(row.get('tags', '')) or str(row.get('Tags', '')),
            'Document Title': row['Title'],
            'Exists?': 'Y',
            'Publication Date': str(row.get('Publication Date', '')),
            'Publication Year': '',
            'Start Year': '',
            'End Year': '',
            'Language': 'EN',
            'Link': row['Link'],
            'Content': text,  # Extracted text content from PDF

            'Description': str(row.get('Short Description', '')),
            'Journal': str(row.get('Journal', '')),

        }

    elif source_type == 'UNResolution':
        document_json = {
            'Code': '',
            'Status': '',
            'Country Name': '',
            'Country Code': '',
            'Category': '',
            'KeyWord to Search': '',
            'Document Title': '',
            'Exists?': 'Y',
            'Publication Date': '',
            'Publication Year': '',
            'Start Year': '',
            'End Year': '',
            'Language': 'EN',
            'Link': row['Link'],
            'Content': text,  # Extracted text content from PDF

            'Resolution No': str(row.get('Resolution No', '')),

        }

    else :
        document_json={}


    # Write the dictionary to a JSON file
    with open(json_path, 'w') as jsonfile:
        json.dump(document_json, jsonfile)
 

In [8]:
def jsonify_if_missing(json_path,clean_text_path,row,source_type):
    
    if not os.path.exists(json_path):
        ##extract text if it is not already present
        create_document_json(json_path,clean_text_path,row,source_type)
    
    else:
        print(json_path,"already exists")

In [9]:
def jsonify_all(metadata_df,missing_only, source_type):
    metadata_df=metadata_df.fillna(np.nan).replace([np.nan], [None])##fixes missing so json works
    
    for index, row in metadata_df.iterrows():

        if source_type == 'National':
           folder="03_National/"+row["Code"].split("-")[0]## update this for more docs
        
        elif source_type == 'SEH':
            folder="00_SEH"## update this for more docs

        elif source_type == 'UNResolution':
            folder="05_UN Resolutions"## update this for more docs
      
        elif source_type == 'Publications':
            folder="02_Publications/UNDP"## update this for more docs

        else : 
             folder =''

        clean_text_folder="../02_Output/01_Cleaned-Text/"+folder ##later update this to the manual folder
        json_folder="../02_Output/03_Pdf-Jsons/"+folder
        
        if source_type == 'National':
           clean_text_path=clean_text_folder+'/'+row["Code"]+"-clean.txt"
        
        elif source_type == 'UNResolution':
             clean_text_path=clean_text_folder+'/'+str(row["Resolution No"]).replace('/',':')+".pdf.txt"

        elif source_type == 'SEH':
             if row["Title"] != None:
                 clean_text_path=clean_text_folder+'/'+row["Title"]+".txt"

        elif source_type == 'Publications':
             if row["Title"] != None:
                clean_text_path=clean_text_folder+'/'+row["Title"]+".txt"

        else : 
             clean_text_path =''

        if source_type == 'National':
            json_path=json_folder+'/'+row["Code"]+".json"

        elif source_type == 'UNResolution':
            json_path=json_folder+'/'+str(row["Resolution No"]).replace('/',':')+".json"

        elif source_type == 'SEH':
             if row["Title"] != None:
                json_path=json_folder+'/'+row["Title"]+".json"

        elif source_type == 'Publications':
             if row["Title"] != None:
                json_path=json_folder+'/'+row["Title"]+".json"

        else : 
             json_path =''
        # print(clean_text_path)
        if os.path.exists(clean_text_path):
            # print(f" Path found {clean_text_path}")

            if not os.path.exists(json_folder):
                # Create the directory
                os.makedirs(json_folder)
                print(json_folder+" directory created!")
            
            if missing_only:
                jsonify_if_missing(json_path,clean_text_path,row,source_type)
            else:
                create_document_json(json_path,clean_text_path,row,source_type)
        else:
            r=''
            # if source_type == 'National':
            #    print("can't jsonify because no clean text",row["Code"])
            # elif source_type == 'SEH':
            #      print("can't jsonify because no clean text")


In [ ]:
# jsonify_all(metadata_df, True, 'National') #For National Documents

# jsonify_all(metadata_df_un_resolution, True, 'UNResolution') #For United Nations Resolution

# For publications and SEH
metadata_file_list = ['(Old) Mapping Journal articles - UNV Sustainable Energy Hub.xlsx',
                      'Energy Document Catalogue.xlsx',
                      'Energy Journal Article Compilation.xlsx',
                      'UNDP Energy CO Press Releases.xlsx']

category = ['SEH',
            'Publications',
            'Publications',
            'Publications']

for metadata_file, category in zip(metadata_file_list, category):
    print(f"checking {metadata_file}")
    print(f"checking {category}")

    metadata_df_file = pd.read_excel(f'../01_Input/00_Metadata/{metadata_file}')

    jsonify_all(metadata_df_file, True, category)


 

<h3>Merge JSONs into one document.json for llm embeddings</h3>

In [79]:
def merge_json_files(folder_path, output_file):
    # Initialize an empty list to store JSON data
    all_data = []
    
    # Traverse through the folder and its subfolders
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            # Check if the file is a JSON file
            if filename.endswith('.json'):
                file_path = os.path.join(root, filename)
                # Read JSON data from the file
                with open(file_path, 'r') as f:
                    try:
                        json_data = json.load(f)
                        all_data.append(json_data)
                    except Exception as e:
                        print(f"Error reading file '{file_path}': {e}")
    
    # Write the merged JSON data to the output file
    with open(output_file, 'w') as f:
        json.dump(all_data, f, indent=4)
# Example usage:
source_folder = '../02_Output/03_Pdf-Jsons'
folder_path = '../02_Output'
output_file = f'{folder_path}/04_Doc_Jsons_Merge/documents_all.json'
merge_json_files(source_folder, output_file)


-TODO

<p> https://sci-hub.se/ for science direct pdfs </p>
<p> Rename Create_ 04_Journal  folder </p>